## Setup

In [1]:
# Standard modules
import pandas as pd
import numpy as np
import time
from datetime import datetime
import thefuzz
from thefuzz import process

import geopandas as gpd
import geopy as gp
import shapely as shp
from shapely.geometry import LineString
import pyproj # Used to convert coordinate projections

from sqlalchemy import create_engine, text
from os import environ
import mysql.connector

## Roadbed processing

### Read in Data

In [3]:
# File name
directory = environ.get("directory")

roadbed_file = f"{directory}01_data/NYC Street Centerline (CSCL)/geo_export_15a738be-7dbc-4a0f-826d-9bd8517b040a.shp"

# Read in the shape file
rdGDF = gpd.read_file(roadbed_file)

# Project the geometry to a distance based CRS (measured in feet)
rdGDF = rdGDF.to_crs(2263)

### Edit street names to merge with sign data

In [11]:
# Replace frequent errors in street name strings
## Create dictionary to replace values in listed columns
replDict = {'AVENUE':'AVE', 
            'STREET':'ST', 
            'PLACE':'PL', 
            'COURT':'CT', 
            'ROAD':'RD', 
            'DRIVE':'DR', 
            'LANE':'LN', 
            'TERRACE':'TER', 
            'HIGHWAY':'HWY',
            'PARKWAY':'PKWY', 
            'BOULEVARD':'BLVD', 
            'TURNPIKE':'TPKE', 
            'EXPRESSWAY':'EXPY', 
            'SQUARE':'SQ', 
            'CIRCLE':'CIR'}

## Create a regular expression pattern to match whole words
for error, replacement in replDict.items():
    pattern = r'\b{}\b'.format(error)
    rdGDF['full_stree'] = rdGDF['full_stree'].replace(pattern, replacement, regex=True)

# Create a borough name variable
rdGDF['boroname'] = rdGDF['borocode'].map({'1':'Manhattan', '2':'Bronx', '3':'Brooklyn', '4':'Queens', '5':'Staten Island'})


### Create offset lines to represent each curb

In [13]:
# Create a function to produce the offset LineStrings that represent each curb
def create_parallel_lines(line, street_width, side):
    distance = street_width
    curb = line.parallel_offset(distance, side)
    return curb

In [15]:
# Create curb geometry
rdGDF['l_curb'] = rdGDF.apply(lambda row: create_parallel_lines(row['geometry'], row['st_width']/2, 'left'), axis=1)
rdGDF['r_curb'] = rdGDF.apply(lambda row: create_parallel_lines(row['geometry'], row['st_width']/2, 'right'), axis=1)

# Create WKT column for geometry
rdGDF['geometry_wkt'] = rdGDF['geometry'].apply(lambda x: x.wkt)
rdGDF['l_curb_wkt'] = rdGDF['l_curb'].apply(lambda x: x.wkt)
rdGDF['r_curb_wkt'] = rdGDF['r_curb'].apply(lambda x: x.wkt)

In [17]:
# Create a gazetteer of unique street names in the roadbed data - will be used to fuzzy merge with street sign names
rdUni = rdGDF[['full_stree', 'boroname']].drop_duplicates()

# Create an in variable to identify records in the merge
rdUni['in_rd'] = 1

## Street Sign Processing

### Read in Data

In [21]:
# File name
streetSigns_file = f'{directory}01_data/Parking_Regulation_Locations_and_Signs.csv'

# Read in the street signs data
streetSigns_df = pd.read_csv(streetSigns_file)

streetSigns_gdf = gpd.GeoDataFrame(streetSigns_df, geometry=gpd.points_from_xy(streetSigns_df.sign_x_coord, streetSigns_df.sign_y_coord), crs="EPSG:2263")

### Edit street names to merge with roadbed 

In [29]:
# Replace frequent errors in street name strings
## Create dictionary to replace values in listed columns
replDict = {'AVENUE':'AVE', 
            'STREET':'ST', 
            'PLACE':'PL', 
            'COURT':'CT', 
            'ROAD':'RD', 
            'DRIVE':'DR', 
            'LANE':' LN', 
            'TERRACE':'TER', 
            'HIGHWAY':'HWY',
            'PARKWAY':'PKWY', 
            'BOULEVARD':'BLVD', 
            'TURNPIKE':'TPKE', 
            'EXPRESSWAY':'EXPY', 
            'SQUARE':'SQ', 
            'CIRCLE':'CIR',
            'BEACH':'BCH',
            '\s+':' '}

## Replace street type with abbreviation in 'on_street' column match the convention in the shapefile
streetSigns_gdf['on_street'] = streetSigns_gdf['on_street'].replace(replDict, regex=True)

<>:19: SyntaxWarning: invalid escape sequence '\s'
<>:19: SyntaxWarning: invalid escape sequence '\s'
/var/folders/mf/8c9q6j514k1_fgpgs5221sch0000gn/T/ipykernel_29413/254649931.py:19: SyntaxWarning: invalid escape sequence '\s'
  '\s+':' '}


### Edit street names to match roadbed

In [31]:
# Create a dataframe of unique street name/borough combinations to merge with the roadbed dataframe
streetSignsUni = streetSigns_gdf[['on_street', 'borough']].drop_duplicates()

streetSignsUni['in_sani'] = 1

In [33]:
# Create a left join of the sanitation sign and roadbed street name/borough dataframes
merge = streetSignsUni.merge(rdUni, how = 'left', left_on = ['on_street', 'borough'], right_on = ['full_stree', 'boroname'])

# Remove observations that joined successfully with the roadbed street name/borough dataframe to make the fuzzy merge faster
streetSigns_tofuzz = merge.loc[merge.in_rd != 1, ['on_street', 'borough']]

# Create an empty DataFrame to store all of the borough level data
streetSigns_xwalk = pd.DataFrame()

# Create a loop to fuzzy merge the remaining street names by borough - limiting by borough creates five smaller Cartesian products, which results in fewer comparisons
for b in streetSigns_tofuzz.borough.unique():
    saniSigns_boro = streetSigns_tofuzz[streetSigns_tofuzz.borough == b].copy()
    rd_boro = rdUni[rdUni.boroname == b]
    saniSigns_boro.loc[:, 'full_stree'] = saniSigns_boro['on_street'].apply(lambda x: thefuzz.process.extractOne(x, rd_boro['full_stree'], scorer = thefuzz.fuzz.ratio)[0])
    streetSigns_xwalk = pd.concat([streetSigns_xwalk, saniSigns_boro], ignore_index = True)

# Append the street names that already had a perfect match in the roadbed data
streetSigns_xwalk = pd.concat((streetSigns_xwalk, merge.loc[merge.in_rd == 1, ['on_street', 'borough', 'full_stree']]))

In [35]:
# Merge the crosswalk to the signs GDF
streetSignsGDF_names = streetSigns_gdf.merge(streetSigns_xwalk, how = 'outer', on = ['on_street', 'borough'])
streetSignsGDF_names['geometry_wkt'] = streetSignsGDF_names['geometry'].apply(lambda x: x.wkt)
streetSignsGDF_names = streetSignsGDF_names.where(pd.notnull(streetSignsGDF_names), None)

## Load data to tables

In [9]:
# Create engine to load data to the table
engine = environ.get("sqlalchemy_engine")

engine = create_engine(engine)

In [ ]:
rdGDF.drop(columns = ['geometry', 'l_curb', 'r_curb']).to_sql('road_stg', con=engine, if_exists='replace', index=False, chunksize=1000)

In [ ]:
streetSignsGDF_names.drop(columns = ['geometry']).to_sql('sign_stg', con=engine, if_exists='replace', index=False, chunksize=1000)